<a href="https://colab.research.google.com/github/Muhammad-Abdullah-Jami/UK_Bank_Stocks_Prediction/blob/main/Yfinance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance


In [ ]:
import yfinance as yf
import pandas as pd

# Fetch data for Standard Chartered (STAN.L)
ticker = "STAN.L"
stock_data = yf.download(ticker, start="2019-01-01", end="2024-01-01")

# Display the first few rows of the data
stock_data.head()



[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,596.000000,607.700012,584.099976,606.400024,501.606232,4442051
2019-01-03,596.599976,610.799988,582.700012,586.700012,485.310608,7359146
2019-01-04,597.400024,611.200012,596.000000,607.799988,502.764252,7283237
2019-01-07,606.900024,609.599976,590.900024,594.599976,491.845459,7184535
2019-01-08,596.799988,623.500000,593.599976,613.000000,507.065643,7575939


In [ ]:
import yfinance as yf

# Fetch data for Standard Chartered (STAN.L)
ticker = "STAN.L"
stock_data = yf.download(ticker, start="2019-01-01", end="2024-01-01")

# Calculate the number of years
start_year = stock_data.index[0].year
end_year = stock_data.index[-1].year
years_of_data = end_year - start_year + 1

print(f"Years of data: {years_of_data}")


[*********************100%***********************]  1 of 1 completed

Years of data: 5


In [ ]:
import yfinance as yf

# Fetch data for Standard Chartered (STAN.L)
ticker = "STAN.L"
stock = yf.Ticker(ticker)

# Get stock metadata
metadata = stock.info

# Display the metadata
metadata


{'address1': '1 Basinghall Avenue',
 'city': 'London',
 'zip': 'EC2V 5DD',
 'country': 'United Kingdom',
 'phone': '44 20 7885 8888',
 'fax': '44 20 7885 9999',
 'website': 'https://www.sc.com',
 'industry': 'Banks - Diversified',
 'industryKey': 'banks-diversified',
 'industryDisp': 'Banks - Diversified',
 'sector': 'Financial Services',
 'sectorKey': 'financial-services',
 'sectorDisp': 'Financial Services',
 'longBusinessSummary': 'Standard Chartered PLC, together with its subsidiaries, provides various banking products and services in Asia, Africa, the Middle East, Europe, and the Americas. The company operates in three segments: Corporate, Commercial & Institutional Banking; Consumer, Private & Business Banking; and Ventures. It offers retail products, such as deposits, mortgages, credit cards, and personal loans; wealth management products and services that include investments, portfolio management, insurance, and wealth advice; and transaction banking services, such as cash mana